分析流程: 讀資料--->資料可視化--->特徵處理--->切分資料--->訓練模型--->預測數據


修正
1.以迴圈方法將缺失值填入中位數避免極端值影響
2.以交叉驗證的方法

In [79]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#load data
dengue_features_train = pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\dengue_features_train.csv")
dengue_features_test =  pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\dengue_features_test.csv")
dengue_labels_train = pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\dengue_labels_train.csv")

In [3]:
#see the shape of training feature
dengue_features_train.shape

(1456, 24)

In [4]:
#see the first 5 rows of training feature
dengue_features_train.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [5]:
#see the shape of test feature
dengue_features_test.shape

(416, 24)

In [6]:
#see the first 5 rows of test feature
dengue_features_test.head()

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.0189,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.0180,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.0015,NaN,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,NaN,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.0568,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1


In [7]:
#see the shape of label
dengue_labels_train.shape

(1456, 4)

In [8]:
#see the first 5 rows of test feature
dengue_labels_train.head()

,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


In [9]:
#merge features and labels of training data
df = pd.merge(dengue_features_train, dengue_labels_train, how='outer', on=dengue_labels_train.columns.tolist()[:-1])
df.head(10)

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6
5,sj,1990,23,1990-06-04,NaN,0.174850,0.254314,0.181743,9.58,299.630000,...,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1,2
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7,4
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1,5
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1,10
9,sj,1990,27,1990-07-02,NaN,0.121550,0.160683,0.202567,14.41,300.154286,...,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1,6


In [10]:
df.shape

(1456, 25)

In [ ]:
sns.pairplot(df)

In [11]:
#see Nan
df.isnull().sum()

city                                       0
year                                       0
weekofyear                                 0
week_start_date                            0
ndvi_ne                                  194
ndvi_nw                                   52
ndvi_se                                   22
ndvi_sw                                   22
precipitation_amt_mm                      13
reanalysis_air_temp_k                     10
reanalysis_avg_temp_k                     10
reanalysis_dew_point_temp_k               10
reanalysis_max_air_temp_k                 10
reanalysis_min_air_temp_k                 10
reanalysis_precip_amt_kg_per_m2           10
reanalysis_relative_humidity_percent      10
reanalysis_sat_precip_amt_mm              13
reanalysis_specific_humidity_g_per_kg     10
reanalysis_tdtr_k                         10
station_avg_temp_c                        43
station_diur_temp_rng_c                   43
station_max_temp_c                        20
station_mi

In [27]:
#將所有遺漏值以中位數方式填入，避免極端值造成的影響
for i in df.columns[4:25]:
    v = df[i].median()
    df[i] = df[i].fillna(value = v)

In [28]:
df.head(10) #檢查填入中位數後的資料樣貌

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6
5,sj,1990,23,1990-06-04,0.128817,0.174850,0.254314,0.181743,9.58,299.630000,...,79.891429,9.58,17.212857,2.100000,28.114286,6.942857,34.4,23.9,39.1,2
6,sj,1990,24,1990-06-11,0.112900,0.092800,0.205071,0.210271,3.48,299.207143,...,82.000000,3.48,17.234286,2.042857,27.414286,6.771429,32.2,23.3,29.7,4
7,sj,1990,25,1990-06-18,0.072500,0.072500,0.151471,0.133029,151.12,299.591429,...,83.375714,151.12,17.977143,1.571429,28.371429,7.685714,33.9,22.8,21.1,5
8,sj,1990,26,1990-06-25,0.102450,0.146175,0.125571,0.123600,19.32,299.578571,...,82.768571,19.32,17.790000,1.885714,28.328571,7.385714,33.9,22.8,21.1,10
9,sj,1990,27,1990-07-02,0.128817,0.121550,0.160683,0.202567,14.41,300.154286,...,81.281429,14.41,18.071429,2.014286,28.328571,6.514286,33.9,24.4,1.1,6


In [14]:
df.isnull().sum() #檢查是否還有遺漏值

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

In [15]:
df.shape

(1456, 25)

In [65]:
#將資料切成train和test資料集
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation

data_train = df.iloc[:,4:24]
data_test = df.iloc[:,24]

train_X, test_X, train_y, test_y = cross_validation.train_test_split(data_train, data_test, test_size = 0.3)

In [125]:
#建立線性回歸模型並將資料餵入
from sklearn.linear_model import Lasso, Ridge
clf = Lasso(alpha=1e0)
clf.fit(train_X, train_y)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [126]:
#用訓練好的模型來預測
y_pred = clf.predict(test_X)
y_pred=y_pred.round()
y_pred

array([ 24.,  22.,  46.,  45.,  24.,  19.,  45.,  28., -10.,  -3.,  32.,
         6.,  24.,  21.,  40.,  34.,  36.,   4.,  25.,  -9.,  -6.,  41.,
        -9.,  12.,  40.,  45.,  -4.,  35.,  29.,  24.,   0.,  36.,  41.,
         1.,  19.,  20.,  34.,  10.,  38.,  41.,   3.,  -1.,  31.,  26.,
        45.,  38.,  23.,   8.,  23.,  -2.,   3.,  23.,  41.,  38.,  14.,
        13.,  36.,  19.,  28.,   3.,   3.,  14.,   7.,  25.,  39.,  33.,
        38.,  34.,  40.,  15.,  15.,  23.,  30.,  18.,  25.,  14.,  45.,
        12.,  14.,  39.,  37.,  34.,  46.,  20.,  45.,  42.,  22.,  -1.,
        21.,  22.,   2.,  17.,  23.,  42.,   7.,  42.,   8.,   9.,  12.,
         1.,   6.,  14.,  21.,   5.,  21.,   4.,  -0.,  11.,   4.,  33.,
        42.,  28.,  14.,  17.,  23.,  24.,  29.,   6.,  25.,  24.,  22.,
        26.,  30.,   8.,   8.,  41.,  41.,  -2.,  17.,  37.,  41.,   0.,
        13.,  11.,  25.,  -2.,  40.,  21.,  40.,  26.,  10.,  43.,  23.,
        23.,  37.,  -1.,  39.,  32.,   9.,  38.,  1

In [127]:
y_pred[y_pred< 0] = 0
y_pred

array([ 24.,  22.,  46.,  45.,  24.,  19.,  45.,  28.,   0.,   0.,  32.,
         6.,  24.,  21.,  40.,  34.,  36.,   4.,  25.,   0.,   0.,  41.,
         0.,  12.,  40.,  45.,   0.,  35.,  29.,  24.,   0.,  36.,  41.,
         1.,  19.,  20.,  34.,  10.,  38.,  41.,   3.,   0.,  31.,  26.,
        45.,  38.,  23.,   8.,  23.,   0.,   3.,  23.,  41.,  38.,  14.,
        13.,  36.,  19.,  28.,   3.,   3.,  14.,   7.,  25.,  39.,  33.,
        38.,  34.,  40.,  15.,  15.,  23.,  30.,  18.,  25.,  14.,  45.,
        12.,  14.,  39.,  37.,  34.,  46.,  20.,  45.,  42.,  22.,   0.,
        21.,  22.,   2.,  17.,  23.,  42.,   7.,  42.,   8.,   9.,  12.,
         1.,   6.,  14.,  21.,   5.,  21.,   4.,  -0.,  11.,   4.,  33.,
        42.,  28.,  14.,  17.,  23.,  24.,  29.,   6.,  25.,  24.,  22.,
        26.,  30.,   8.,   8.,  41.,  41.,   0.,  17.,  37.,  41.,   0.,
        13.,  11.,  25.,   0.,  40.,  21.,  40.,  26.,  10.,  43.,  23.,
        23.,  37.,   0.,  39.,  32.,   9.,  38.,  1

In [128]:
#以MAE評估模型效能
from sklearn.metrics import mean_absolute_error, r2_score
# The coefficients
print('Coefficients: \n', clf.coef_)

# The mean absolute error
print("Mean absolute error: %.2f"
      % mean_absolute_error(test_y, y_pred))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(test_y, y_pred))

Coefficients: 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
  -7.62935643e-05   4.06376290e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   1.73824504e-01   3.19359579e-02  -1.85131674e-01
  -1.73177008e-02   0.00000000e+00  -3.08727571e+00   9.47730706e-01
  -2.05393041e-01   1.54131117e+00   0.00000000e+00   4.23709407e-03]
Mean absolute error: 20.59
Variance score: 0.11


In [129]:
dengue_features_test_df = pd.DataFrame(dengue_features_test) #將要預測的資料轉換成dataframe的格式
dengue_features_test_df.isnull().sum() #檢查是否還有遺漏值

city                                     0
year                                     0
weekofyear                               0
week_start_date                          0
ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_pre

In [130]:
#將所有遺漏值以中位數方式填入，避免極端值造成的影響
for j in dengue_features_test_df.columns[4:]:
    v = dengue_features_test_df[j].median()
    dengue_features_test_df[j] = dengue_features_test_df[j].fillna(value = v)

In [131]:
dengue_features_test_df.head(10) #檢查前10列的缺失值是否成功填入

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,2008-04-29,-0.018900,-0.018900,0.102729,0.091200,78.60,298.492857,...,25.37,78.781429,78.60,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2
1,sj,2008,19,2008-05-06,-0.018000,-0.012400,0.082043,0.072314,12.56,298.475714,...,21.83,78.230000,12.56,15.791429,2.571429,26.071429,5.557143,30.0,22.2,34.3
2,sj,2008,20,2008-05-13,-0.001500,0.088700,0.151083,0.091529,3.66,299.455714,...,4.12,78.270000,3.66,16.674286,4.428571,27.928571,7.785714,32.8,22.8,3.0
3,sj,2008,21,2008-05-20,0.110100,-0.019867,0.124329,0.125686,0.00,299.690000,...,2.20,73.015714,0.00,15.775714,4.342857,28.057143,6.271429,33.3,24.4,0.3
4,sj,2008,22,2008-05-27,0.056800,0.039833,0.062267,0.075914,0.76,299.780000,...,4.36,74.084286,0.76,16.137143,3.542857,27.614286,7.085714,33.3,23.3,84.1
5,sj,2008,23,2008-06-03,-0.044000,-0.030467,0.132000,0.083529,71.17,299.768571,...,22.55,76.557143,71.17,16.667143,2.857143,28.000000,5.171429,32.8,25.0,27.7
6,sj,2008,24,2008-06-10,-0.044300,-0.024925,0.132271,0.159157,48.99,300.062857,...,13.10,76.844286,48.99,17.010000,3.157143,27.400000,6.042857,31.1,23.3,91.7
7,sj,2008,25,2008-06-17,0.110100,0.082150,0.144371,0.116729,30.81,300.484286,...,7.20,76.870000,30.81,17.420000,3.900000,28.757143,6.985714,34.4,24.4,0.3
8,sj,2008,26,2008-06-24,0.010800,0.049900,0.100571,0.117329,8.02,300.601429,...,17.10,77.395714,8.02,17.678571,2.785714,28.657143,6.242857,32.8,23.9,28.7
9,sj,2008,27,2008-07-01,0.072667,0.106660,0.155429,0.164900,17.52,300.497143,...,11.90,78.534286,17.52,17.808571,2.228571,28.457143,4.628571,31.1,25.0,2.9


In [132]:
dengue_features_test_df_new = dengue_features_test_df.iloc[:,4:24] #略過前四欄的字串資料

In [133]:
#進行預測
result = clf.predict(dengue_features_test_df_new)
result = result.round()
result

array([ 29.,  27.,  30.,  34.,  36.,  37.,  34.,  39.,  42.,  40.,  39.,
        40.,  42.,  42.,  42.,  46.,  44.,  43.,  46.,  43.,  40.,  40.,
        42.,  40.,  38.,  34.,  38.,  37.,  34.,  35.,  33.,  26.,  28.,
        30.,  22.,  31.,  20.,  27.,  26.,  27.,  23.,  25.,  23.,  23.,
        23.,  24.,  18.,  17.,  18.,  21.,  25.,  30.,  31.,  25.,  29.,
        27.,  31.,  37.,  36.,  38.,  40.,  41.,  44.,  42.,  43.,  44.,
        45.,  45.,  43.,  45.,  48.,  41.,  44.,  44.,  44.,  47.,  45.,
        44.,  44.,  40.,  37.,  36.,  36.,  35.,  37.,  35.,  34.,  31.,
        29.,  27.,  25.,  26.,  25.,  26.,  25.,  32.,  33.,  29.,  29.,
        35.,  35.,  34.,  27.,  32.,  37.,  41.,  33.,  37.,  38.,  44.,
        41.,  43.,  40.,  38.,  43.,  46.,  40.,  37.,  41.,  48.,  41.,
        46.,  45.,  47.,  43.,  44.,  40.,  39.,  41.,  40.,  43.,  41.,
        41.,  27.,  32.,  30.,  20.,  26.,  26.,  25.,  19.,  26.,  26.,
        24.,  22.,  24.,  25.,  21.,  20.,  19.,  1

In [134]:
#讀入比賽格式
dengue_submission = pd.read_csv("C:\\Users\\user\\Desktop\\python workspace\\drivendata\\submission_format.csv")

In [135]:
#將預測結果寫入比賽格式
dengue_submission['total_cases'] = result
dengue_submission['total_cases'] = dengue_submission['total_cases'].astype(int)
dengue_submission.to_csv('submission_format_v3.csv', index = False)

In [136]:
dengue_submission.head()

,city,year,weekofyear,total_cases
0,sj,2008,18,29
1,sj,2008,19,27
2,sj,2008,20,30
3,sj,2008,21,34
4,sj,2008,22,36
